In [10]:
pip install nrel-pysam

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 36.8 MB 564 kB/s eta 0:00:01
     |████████████████████████████████| 11.3 MB 11.3 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 17.7 MB/s eta 0:00:01
     |████████████████████████████████| 345 kB 9.2 MB/s eta 0:00:01
     |████████████████████████████████| 14.0 MB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 505 kB 9.2 MB/s eta 0:00:01
     |████████████████████████████████| 164 kB 9.6 MB/s eta 0:00:01
     |████████████████████████████████| 120 kB 9.6 MB/s eta 0:00:01
     |████████████████████████████████| 121 kB 9.3 MB/s eta 0:00:01
     |████████████████████████████████| 66 kB 9.8 MB/s  eta 0:00:01
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [12]:
%pip install openqaoa

%pip install pandas
%pip install requests
%pip install geojson
%pip install geopandas

%pip install cplex

%pip install qiskit
%pip install fiona
%pip install pyogrio
%pip install pyproj
%pip install rtree 
%pip install shapely
%pip install numpy
%pip install geodatasets
%pip install geoplot
%pip install matplotlib
%pip install qiskit[visualization]
%pip install folium
%pip install libpysal
%pip install dwave-ocean-sdk
%pip install networkx


%pip install scikit-learn
%pip install qiskit[optimization]

%pip install more-itertools
%matplotlib inline

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 488 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 288 kB 12.7 MB/s eta 0:00:01
     |████████████████████████████████| 40 kB 24.7 MB/s eta 0:00:01
     |████████████████████████████████| 142 kB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 312 kB 11.6 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 2.9 MB/s eta 0:00:011
     |████████████████████████████████| 7.8 MB 8.4 MB/s eta 0:00:01
     |████████████████████████████████| 633 kB 12.2 MB/s eta 0:00:01
     |████████████████████████████████| 5.7 MB 13.3 MB/s eta 0:00:01
     |████████████████████████████████| 30.3 MB 3.7 MB/s eta 0:00:012
     |████████████████████████████████| 1.6 MB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 6.5 MB 3.7 MB/s eta 0:00:01
  distutils: /private/var/folders

Matplotlib is building the font cache; this may take a moment.


In [13]:
print("hi!")

hi!
